# The Cars of eBay Germany - Practice Cleaning a Dirty Dataset

In this project, we are exploring, and primarily, cleaning a dataset consisting of cars found on *ebay Kleinanzeigen*, a classified section of the German eBay website. 

The data consists of plenty of fields such as, but not limited to:

- name: The name of the car
- seller: Whether the seller is private or a dealer
- price: The price on the add to sell the car
- yearOfRegistration: The year the car was first registered
- model: The car model name

As mentioned earlier, the purpose of this project is to clean this data. As we will see, there are many aspects of it that need to be dealt with before anyone form of analysis can be performed. 

We'll start with importing the data, the necessary libraries, and looking at the data.

In [42]:
import numpy as np
import pandas as pd

autos = pd.read_csv("autos.csv", encoding = "Latin-1")

### Exploring the Data

The info() method of the DataFrame class displays the names of the columns as well as the number of non-null values and the types of data in those fields. The fields either consist of the int data type or object data type, which are typically strings. 

More importantly, we can see how the max number of non-null values is 50,000, which makes sense because that's how fields our data contains. Some of the columns however, such as **vehicleType**, **gearbox**, and **model** have less, meaning some values are *Null*.

In [43]:
autos.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [44]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Looking at the first 5 rows of the data, there are some things we can notice right off the bat:

1. It may not be a big deal, but the seller is either "private" or "dealer". In the column however, we can see it says "privat" instead.
2. The dateCrawled and other date columns could be a string, even though it should be a DateTime.
3. The price column contains string values even though they should be integers.
4. Just like the price column, the odometer column has the same issue.
5. The names of the columns are in camelCase instead of snake_case, so we must fix that as well.

We'll start with changing the column names to Snake Case from Camel Case.

In [45]:
print(autos.columns)

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [46]:
auto_columns = autos.columns

mapping_changes = {
    "yearOfRegistration": "registration_year",
    "monthOfRegistration": "registration_month",
    "notRepairedDamage": "unrepaired_damage",
    "dateCreated": "ad_created",
    "dateCrawled": "date_crawled",
    "name": "name",
    "abtest": "ab_test",
    "seller": "seller",
    "gearbox": "gearbox",
    "model": "model",
    "odometer": "odometer",
    "brand": "brand",
    "price": "price",
    "offerType": "offer_type",
    "vehicleType": "vehicle_type",
    "powerPS": "power_ps",
    "fuelType": "fuel_type",
    "nrOfPictures": "num_of_pictures",
    "postalCode": "postal_code",
    "lastSeen": "last_seen"
}

''''This line converts the list to a Series and then access the map method to 
change the names.'''
autos.columns = pd.Series(auto_columns).map(mapping_changes) 

In [47]:
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


Here are the changes that were made in the columns names:

- yearOfRegistration, monthOfRegistration, notRepairedDamage, and dateCreated were all changed to different names because the names were too long and could cause confusion
- The rest of the columns that had more than one word in them were converted to snake case to keep with Python conventions

The next step of our cleaning process consists of determining if any text columns have only one value in them and to convert fields to numeric that are currently objects.

We want to see if a field only has one value, because if it does, this is an indication that the field is practicialy uninformative. Removing it would not cause any significant changes to the data. 

In [48]:
print(autos.describe(include = "all"))

               date_crawled         name  seller offer_type  price ab_test  \
count                 50000        50000   50000      50000  50000   50000   
unique                48213        38754       2          2   2357       2   
top     2016-03-23 19:38:20  Ford_Fiesta  privat    Angebot     $0    test   
freq                      3           78   49999      49999   1421   25756   
mean                    NaN          NaN     NaN        NaN    NaN     NaN   
std                     NaN          NaN     NaN        NaN    NaN     NaN   
min                     NaN          NaN     NaN        NaN    NaN     NaN   
25%                     NaN          NaN     NaN        NaN    NaN     NaN   
50%                     NaN          NaN     NaN        NaN    NaN     NaN   
75%                     NaN          NaN     NaN        NaN    NaN     NaN   
max                     NaN          NaN     NaN        NaN    NaN     NaN   

       vehicle_type  registration_year  gearbox      power_ps  

The describe() method lets us look at the data in much more detail. Here are some things that stand out:

- The seller and offer_type column practically only have 1 value, that being private and Angebot respectively. We can remove those columns as they will not provide any useful information in our analysis
- The price and odometer columns are not in numeric form

Let's fix the price and odometer columns first.

In [49]:
#Remove any of the "bad characters" in the string 
def clean_column(string):
    bad_chars = ["$", ",", "km"]
    for c in bad_chars:
        string = string.replace(c, "")
    return string   

cleaned_prices = []
cleaned_odometers = []

for price in autos["price"]:
    cleaned_price = clean_column(price)
    cleaned_prices.append(cleaned_price)

for odo in autos["odometer"]:
    cleaned_odo = clean_column(odo)
    cleaned_odometers.append(cleaned_odo)

#Convert to a Series type and then an int type
cleaned_prices = pd.Series(cleaned_prices).astype(int)
cleaned_odometers = pd.Series(cleaned_odometers).astype(int)

#Put the new column values into the dataset
autos["price"] = cleaned_prices
autos["odometer"] = cleaned_odometers

autos.rename({"odometer" : "odometer_km"}, inplace = True, axis = 1)

In [50]:
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


NICE! The price and odometer columns have been successfully converted to integer type columns.

### Exploration Cont.

So we got this far. Now we need to continue exploring the data and clean more inconsistencies/abnormalities with the data. First, we will check the price and odometer_km columns to see if there are any outliers.

In [51]:
print("Unique price values: {}".format(autos["price"].unique().shape[0]))
print("Unique odometer values: {}".format(autos["odometer_km"].unique().shape[0]))

Unique price values: 2357
Unique odometer values: 13


In [52]:
print("Price Description")
print(autos["price"].describe())
print("\n")
print("Odometer Description")
print(autos["odometer_km"].describe())
print("\n")

Price Description
count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


Odometer Description
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64




Looking at the Mins and Maxs:

- For price, the minimum is 0 and the maximum is 100,000,000 which makes absolutely no sense 
- For odometer, the minimum is 50,000 km and the maximum is 150,000 km, which isn't a major deal

Let's remove the outliers of the price column.

In [63]:
autos = autos[(autos["price"] > 0) & (autos["price"] < 100000000)]

In [66]:
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,num_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


### Time to Clean Time Values

So far, we have converted the columns names from camel case to snake case. We have also converted the price and odometer_km columns from strings to integers. Then, we removed outliers from the price column of the data so we have more reasonable records to work with. 

Now, we're going to deal with the time values in the data, specifically in these columns:

- date_crawled
- last_seen
- ad_created

For these three, we must first convert them from a string to a numerical representation.

In [73]:
print(autos["date_crawled"].str[:10].value_counts(normalize = True, dropna = False).sort_index())

2016-03-05    0.025320
2016-03-06    0.014039
2016-03-07    0.036003
2016-03-08    0.033327
2016-03-09    0.033101
2016-03-10    0.032174
2016-03-11    0.032566
2016-03-12    0.036930
2016-03-13    0.015665
2016-03-14    0.036538
2016-03-15    0.034274
2016-03-16    0.029601
2016-03-17    0.031639
2016-03-18    0.012907
2016-03-19    0.034768
2016-03-20    0.037876
2016-03-21    0.037403
2016-03-22    0.033018
2016-03-23    0.032216
2016-03-24    0.029334
2016-03-25    0.031598
2016-03-26    0.032195
2016-03-27    0.031083
2016-03-28    0.034850
2016-03-29    0.034130
2016-03-30    0.033677
2016-03-31    0.031845
2016-04-01    0.033677
2016-04-02    0.035468
2016-04-03    0.038597
2016-04-04    0.036518
2016-04-05    0.013092
2016-04-06    0.003170
2016-04-07    0.001400
Name: date_crawled, dtype: float64


For the date_crawled column, the amount of dates are distributed pretty evenly with the exception of **2016-04-07** and **2016-04-06**, which are significantly lower in presence than the rest of the dates.

In [74]:
print(autos["ad_created"].str[:10].value_counts(normalize = True, dropna = False).sort_index())

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033162
2016-03-10    0.031886
2016-03-11    0.032895
2016-03-12    0.036765
2016-03-13    0.017003
2016-03-14    0.035180
2016-03-15    0.034006
2016-03-16    0.030116
2016-03-17    0.031289
2016-03-18    0.013586
2016-03-19    0.033677
2016-03-20    0.037938
2016-03-21 

Looking at the ad_created column, we can see that most of the dates are situated from **2016-03-09** onwards. Dates before that are not much present in the data. This suggests that multiple ads were created in this day range.

In [75]:
print(autos["last_seen"].str[:10].value_counts(normalize = True, dropna = False).sort_index())

2016-03-05    0.001070
2016-03-06    0.004323
2016-03-07    0.005393
2016-03-08    0.007431
2016-03-09    0.009613
2016-03-10    0.010663
2016-03-11    0.012372
2016-03-12    0.023796
2016-03-13    0.008893
2016-03-14    0.012598
2016-03-15    0.015871
2016-03-16    0.016447
2016-03-17    0.028078
2016-03-18    0.007349
2016-03-19    0.015830
2016-03-20    0.020647
2016-03-21    0.020647
2016-03-22    0.021367
2016-03-23    0.018527
2016-03-24    0.019762
2016-03-25    0.019206
2016-03-26    0.016797
2016-03-27    0.015645
2016-03-28    0.020873
2016-03-29    0.022355
2016-03-30    0.024764
2016-03-31    0.023796
2016-04-01    0.022788
2016-04-02    0.024928
2016-04-03    0.025196
2016-04-04    0.024476
2016-04-05    0.124766
2016-04-06    0.221824
2016-04-07    0.131909
Name: last_seen, dtype: float64


The last_seen column values are distributed pretty evenly. Not too many low occurrences nor high occurrences. 

In [76]:
print(autos["registration_year"].describe())

count    48579.000000
mean      2004.753000
std         88.631663
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64


The minimum and maximum registration years are 1000 and 9999 respectively. This is absolutely impossible, so we will have to do something about it to make it make more sense.

Looking at this, we can determine that any year past 2016 is inaccurate since it would be impossible to register a car AFTER a listing has been seen. However, it is more challenging task to find the earliest valid year within the dataset. 

To combat this, we will count the number of listings that fall out of the 1900 - 2016 year range and proceed from there.

In [90]:
unique_years = autos[(autos["registration_year"] > 1900) 
                     & (autos["registration_year"] <= 2016)]["registration_year"].unique()
print(sorted(unique_years))

[1910, 1927, 1929, 1931, 1934, 1937, 1938, 1939, 1941, 1943, 1948, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016]


As we can see from the array of unique registration_year values above, the lowest acceptable value for registration_year is 1910 while the highest is 2016. 

Now, we will remove all records from the dataset that fall out of this range.

In [93]:
autos = autos[(autos["registration_year"] > 1910) 
              & (autos["registration_year"] <= 2016)]

autos["registration_year"].value_counts(normalize = True)

2000    0.067598
2005    0.062886
1999    0.062093
2004    0.057895
2003    0.057809
2006    0.057210
2001    0.056481
2002    0.053247
1998    0.050613
2007    0.048771
2008    0.047443
2009    0.044680
1997    0.041788
2011    0.034763
2010    0.034034
1996    0.029408
2012    0.028059
1995    0.026302
2016    0.026131
2013    0.017199
2014    0.014222
1994    0.013472
1993    0.009103
2015    0.008396
1992    0.007946
1990    0.007432
1991    0.007261
1989    0.003727
1988    0.002892
1985    0.002056
          ...   
1966    0.000471
1977    0.000471
1976    0.000450
1969    0.000407
1975    0.000386
1965    0.000364
1964    0.000257
1963    0.000171
1961    0.000129
1959    0.000129
1956    0.000086
1958    0.000086
1937    0.000086
1962    0.000086
1950    0.000064
1954    0.000043
1941    0.000043
1934    0.000043
1957    0.000043
1951    0.000043
1955    0.000043
1931    0.000021
1953    0.000021
1943    0.000021
1938    0.000021
1939    0.000021
1927    0.000021
1929    0.0000

From the distribution of when the cars were first registered, the data looks to be left skewed. Most of the values fall in the later years of the data, starting around 1985. 

Now, we are going to explore the prices of the top 20 card brands on Germany's eBay by using aggregation.

In [126]:
brand_selection = autos["brand"].value_counts()[:20].index.tolist()
brand_selection

['volkswagen',
 'bmw',
 'opel',
 'mercedes_benz',
 'audi',
 'ford',
 'renault',
 'peugeot',
 'fiat',
 'seat',
 'skoda',
 'nissan',
 'mazda',
 'smart',
 'citroen',
 'toyota',
 'hyundai',
 'sonstige_autos',
 'volvo',
 'mini']

I chose to use the top 20 car brands because with the numerous amount of those brands bought, an analysis will be that much more accurate. We see that **Volkswagen** and **BMW** are the top 2 with **Volvo** and **Mini** being #19 and #20.

In [141]:
brand_prices = {}

for brand in brand_selection[:6]:
    autos_with_selected_brand = autos[autos["brand"] == brand]
    mean = autos_with_selected_brand["price"].mean()
    brand_prices[brand] = mean

for b in brand_prices:
    print(b,": ", brand_prices[b])

audi :  9336.687453600594
opel :  5434.537542322247
volkswagen :  6729.81956411556
bmw :  8571.480147917478
ford :  7456.547932618683
mercedes_benz :  30829.021092362345


**Citroen** has the most outstanding prices on their cars, with an average price of \\$45,486, with **Sonstige Autos** coming in at second and **Mercedes Benz** coming at third. All other cars seem to be much lower in price, with **Fiat** and **Renault** at the lowest.

Looking at them in the scope of being top brands, The top 6 brands are structured in this way:

- Audi, BMW, and Mercedes Benz are more expensive
- Ford and Opel are less expensive
- Volkswagen is in between

For these top 6 brands, we will look at their average milage and see if there is a link to their mean prices. 

In [142]:
#Create a dictionary, aggregate for brand, and calculate average mileage

brand_mileage = {}

for brand in brand_selection[:6]:
    autos_with_selected_brand = autos[autos["brand"] == brand]
    mean = autos_with_selected_brand["odometer_km"].mean()
    brand_mileage[brand] = mean

#Convert both brand_prices and brand_mileage to Series objects
prices_series = pd.Series(brand_prices)
mileage_series = pd.Series(brand_mileage)

#Create a dataframe using the new Series objects
df = pd.DataFrame(prices_series, columns = ["mean_prices"])
df["average_mileage"] = mileage_series

df

,mean_prices,average_mileage
audi,9336.687454,129157.386785
bmw,8571.480148,132575.905021
ford,7456.547933,124243.491577
mercedes_benz,30829.021092,130792.628774
opel,5434.537542,129315.873332
volkswagen,6729.819564,128713.634060


BMW has the highest average mileage for their listed cars, but they have the third highest average price. Mercedes Benz has the second highest average mileage for their listed cars, but they are the most expensive on average. 

Looking at these, there does not seem to be any correlation between price and mileage. 

### Conclusion

In this project, our goal was not to analyze, but to clean and make our dataset more readable and user friendly. 

We did this by changing the column names appropriately, removing non-digit characters from columns and converting them to numeric fields, removed outliers from the price column, extracted dates from some columns, and removed rows that fell out of the registration year range.

Other things I will do and update this project with are:

- Translate some of the german word values found in the dataset (e.g. automatik and manuell)
- Convert the dates into uniform integer data
- Find the most common brand/model combinations
- Determine how much cheaper cars are with damage than non-damaged

I hope you enjoyed exploring through this project. Thank you for reading!